In [13]:
import ipywidgets as widgets
import numpy as np
import keras

from IPython import display as nb

In [14]:
packed_model = keras.models.load_model("../model/2024-10-29_00-09-51.keras")
packed_model.layers

[<WordPieceTokenizer name=word_piece_tokenizer_1, built=False>,
 <StringLookup name=string_lookup_1, built=False>,
 <Sequential name=sequential_2, built=True>]

In [15]:
tokenizer = packed_model.layers[0]
label_lookup = packed_model.layers[1]

model = packed_model.layers[2]
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 64, 100)        │        23,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 64)             │        34,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 6)              │           390 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │            28 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 172,400 (673.44 KB)

 Trainable params: 57,466 (224.48 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 114,934 (448.96 KB)

In [16]:
w_text = widgets.Text(placeholder="Task")
w_text.continuous_update = False
nb.display(w_text)

def on_submit(value):
    nb.clear_output()
    nb.display(w_text)

    task = value["new"]
    print(f"Input: {task}")

    task_vec = tokenizer(task).numpy()
    print(f"Vectorized input: {tokenizer.detokenize(task_vec)}")
    print(f"{task_vec}\n")
    task_vec = np.reshape(task_vec, (1, len(task_vec)))

    prediction = model.predict(task_vec)[0]
    print(f"Prediction vector: {prediction}")

    label_vocab = label_lookup.get_vocabulary()
    for label, probability in zip(label_vocab, prediction):
        percentage = probability * 100
        print(f"\t{label}: {percentage:.5f}%")

    best = label_vocab[np.argmax(prediction)]
    print(f"Prediction: {best}")

w_text.observe(on_submit, "value")


Text(value='watch television', continuous_update=False, placeholder='Task')

Input: watch television
Vectorized input: watch television [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[ 29 111  45  93  26  40  95 226 222 116   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Prediction vector: [0.5503385  0.43318883 0.00265498 0.01381775]
	not-urgent important: 55.03385%
	not-urgent not-important: 43.31888%
	urgent important: 0.26550%
	urgent not-important: 1.38177%
Prediction: not-urgent important
